In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
# for dirname, _, filenames in os.walk('/kaggle/input'):
#     for filename in filenames:
#         print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [2]:
# The model is the DF model by Sirinam et al

from keras.models import Model
from keras.layers import Dense
from keras.layers import Input
from keras.layers import GlobalAveragePooling2D, GlobalMaxPooling2D, Reshape, Dense, multiply, Permute, Concatenate, Conv2D, Add, Activation, Lambda
from keras import backend as K
from keras.activations import sigmoid

from keras.layers import Activation
from keras.layers import ELU
from keras.layers import Conv1D
from keras.layers import MaxPooling1D
from keras.layers import Dropout
from keras.layers.core import Flatten
import tensorflow as tf
import keras.layers as KL
import keras.models as KM
from keras.backend import set_session
config = tf.compat.v1.ConfigProto()
config.gpu_options.allow_growth = True  # dynamically grow the memory used on the GPU
config.log_device_placement = False  # to log device placement (on which device the operation ran)
sess = tf.compat.v1.Session(config=config)
set_session(sess)  # set this TensorFlow session as the default session for Keras


def DF(input_shape=None, emb_size=None):
    # -----------------Entry flow -----------------
    input_data = Input(shape=input_shape)

    filter_num = ['None', 32, 64, 128, 256]
    kernel_size = ['None', 8, 8, 8, 8]
    conv_stride_size = ['None', 1, 1, 1, 1]
    pool_stride_size = ['None', 4, 4, 4, 4]
    pool_size = ['None', 8, 8, 8, 8]

    model = Conv1D(filters=filter_num[1], kernel_size=kernel_size[1],
                   strides=conv_stride_size[1], padding='same', name='block1_conv1')(input_data)
    model = ELU(alpha=1.0, name='block1_adv_act1')(model)

    model = Conv1D(filters=filter_num[1], kernel_size=kernel_size[1],
                   strides=conv_stride_size[1], padding='same', name='block1_conv2')(model)
    model = ELU(alpha=1.0, name='block1_adv_act2')(model)
    model = MaxPooling1D(pool_size=pool_size[1], strides=pool_stride_size[1],
                         padding='same', name='block1_pool')(model)
    model = Dropout(0.1, name='block1_dropout')(model)

    model = Conv1D(filters=filter_num[2], kernel_size=kernel_size[2],
                   strides=conv_stride_size[2], padding='same', name='block2_conv1')(model)
    model = Activation('relu', name='block2_act1')(model)
    model = Conv1D(filters=filter_num[2], kernel_size=kernel_size[2],
                   strides=conv_stride_size[2], padding='same', name='block2_conv2')(model)
    model = Activation('relu', name='block2_act2')(model)
    model = MaxPooling1D(pool_size=pool_size[2], strides=pool_stride_size[3],
                         padding='same', name='block2_pool')(model)
    model = Dropout(0.1, name='block2_dropout')(model)

    model = Conv1D(filters=filter_num[3], kernel_size=kernel_size[3],
                   strides=conv_stride_size[3], padding='same', name='block3_conv1')(model)
    model = Activation('relu', name='block3_act1')(model)
    model = Conv1D(filters=filter_num[3], kernel_size=kernel_size[3],
                   strides=conv_stride_size[3], padding='same', name='block3_conv2')(model)
    model = Activation('relu', name='block3_act2')(model)
    model = MaxPooling1D(pool_size=pool_size[3], strides=pool_stride_size[3],
                         padding='same', name='block3_pool')(model)
    model = Dropout(0.1, name='block3_dropout')(model)

    model = Conv1D(filters=filter_num[4], kernel_size=kernel_size[4],
                   strides=conv_stride_size[4], padding='same', name='block4_conv1')(model)
    model = Activation('relu', name='block4_act1')(model)
    model = Conv1D(filters=filter_num[4], kernel_size=kernel_size[4],
                   strides=conv_stride_size[4], padding='same', name='block4_conv2')(model)
    model = Activation('relu', name='block4_act2')(model)
    model = MaxPooling1D(pool_size=pool_size[4], strides=pool_stride_size[4],
                         padding='same', name='block4_pool')(model)

    output = Flatten()(model)

    dense_layer = Dense(emb_size, name='FeaturesVec')(output)
    shared_conv2 = Model(inputs=input_data, outputs=dense_layer)
    return shared_conv2


2023-01-22 18:25:56.807241: I tensorflow/core/util/util.cc:169] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2023-01-22 18:25:57.986884: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-01-22 18:25:58.856726: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1532] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 22310 MB memory:  -> device: 0, name: NVIDIA GeForce RTX 3090, pci bus id: 0000:9c:00.0, compute capability: 8.6


In [ ]:
import os
import random
import numpy as np
import pickle
import keras.backend as K
from keras.models import Model
from keras.layers import Input, Lambda, Dot
from keras import optimizers
from keras.callbacks import CSVLogger
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'
from keras.backend import set_session
import tensorflow as tf
# config = tf.compat.v1.ConfigProto
# config.gpu_options.allow_growth = True  # dynamically grow the memory used on the GPU
# config.log_device_placement = False  # to log device placement (on which device the operation ran)
# sess = tf.Session(config=config)
# set_session(sess)  # set this TensorFlow session as the default session for Keras

# This is tuned hyper-parameters
alpha = 0.1
batch_size_value = 128
emb_size = 64
number_epoch = 30

description = 'Triplet_Model'
Training_Data_PATH = '/root/dataset/dataset/extracted_AWF775/' 
print(Training_Data_PATH)
print("with parameters, Alpha: %s, Batch_size: %s, Embedded_size: %s, Epoch_num: %s"%(alpha, batch_size_value, emb_size, number_epoch))


alpha_value = float(alpha)
print(description)

# ================================================================================
# This part is to prepare the files' index for geenrating triplet examples
# and formulating each epoch inputs

# Extract all folders' names
dirs = sorted(os.listdir(Training_Data_PATH))

# Each given folder name (URL of each class), we assign class id
# e.g. {'adp.com' : 23, ...}
name_to_classid = {d:i for i,d in enumerate(dirs)}

# Just reverse from previous step
# Each given class id, show the folder name (URL of each class)
# e.g. {23 : 'adp.com', ...}
classid_to_name = {v:k for k,v in name_to_classid.items()}

num_classes = len(name_to_classid)
print("number of classes: "+str(num_classes))

# Each directory, there are n traces corresponding to the identity
# We map each trace path with an integer id, then build dictionaries
# We are mapping
#   path_to_id and id_to_path
#   classid_to_ids and id_to_classid

# read all directories
# c is class
# name_to_classid.items() contains [(directory, classid), ('slickdeals.net', 547), ...]

trace_paths = {c:[directory + "/" + img for img in sorted(os.listdir(Training_Data_PATH + directory))]
         for directory,c in name_to_classid.items()}
# trace_paths --> {0: ['104.com.tw/104.com.tw_0001.pkl', '104.com.tw/104.com.tw_0002.pkl',...] ,....}

# retreive all traces
# to create the list of all traces paths
all_traces_path = []
for trace_list in trace_paths.values():
    all_traces_path += trace_list
# all_trace_path --> ['104.com.tw/104.com.tw_0001.pkl', '104.com.tw/104.com.tw_0002.pkl',...]
# len(all_trace_path = num_class * num_examples e.g. 700 * 25

# map to integers
# just map each path to sequence of ID (from 1 to len(all_trace_path)
path_to_id = {v: k for k, v in enumerate(all_traces_path)}
# path_to_id --> {'chron.com/chron.com_0006.pkl': 1185, 'habrahabr.ru/habrahabr.ru_0001.pkl': 2680, ...}
id_to_path = {v: k for k, v in path_to_id.items()}
# id_to_path --> {0: '104.com.tw/104.com.tw_0001.pkl', 1: '104.com.tw/104.com.tw_0002.pkl', ...}

# build mapping between traces and class
classid_to_ids = {k: [path_to_id[path] for path in v] for k, v in trace_paths.items()}
# classid_to_ids --> {0: [0, 1, 2, 3, 4, 5, 6, 7, 8, 9], 1: [10, 11, 12, 13, 14, 15, 16, 17, 18, 19],...]
id_to_classid = {v: c for c, traces in classid_to_ids.items() for v in traces}
# id_to_classid --> {0: 0, 1: 0, 2: 0, 3: 0, 4: 0, 5: 0, 6: 0, 7: 0, 8: 0, 9: 0, 10: 1,...]

# open trace
all_traces = []
for path in id_to_path.values():
    each_path = Training_Data_PATH + path
    with open(each_path, 'rb') as handle:
        each_trace = pickle.load(handle,encoding='iso-8859-1')
    all_traces += [each_trace]

all_traces = np.vstack((all_traces))
all_traces = all_traces[:, :, np.newaxis]
print("Load traces with ",all_traces.shape)
print("Total size allocated on RAM : ", str(all_traces.nbytes / 1e6) + ' MB')

def build_pos_pairs_for_id(classid): # classid --> e.g. 0
    traces = classid_to_ids[classid]
    # pos_pairs is actually the combination C(10,2)
    # e.g. if we have 10 example [0,1,2,...,9]
    # and want to create a pair [a, b], where (a, b) are different and order does not matter
    # e.g. [(0, 1), (0, 2), (0, 3), (0, 4), (0, 5), (0, 6), (0, 7), (0, 8), (0, 9),
    # (1, 2), (1, 3), (1, 4), (1, 5), (1, 6), (1, 7), (1, 8), (1, 9)...]
    # C(10, 2) = 45
    pos_pairs = [(traces[i], traces[j]) for i in range(len(traces)) for j in range(i+1, len(traces))]
    random.shuffle(pos_pairs)
    return pos_pairs

def build_positive_pairs(class_id_range):
    # class_id_range = range(0, num_classes)
    listX1 = []
    listX2 = []
    for class_id in class_id_range:
        pos = build_pos_pairs_for_id(class_id)
        # -- pos [(1, 9), (0, 9), (3, 9), (4, 8), (1, 4),...] --> (anchor example, positive example)
        for pair in pos:
            listX1 += [pair[0]] # identity
            listX2 += [pair[1]] # positive example
    perm = np.random.permutation(len(listX1))
    # random.permutation([1,2,3]) --> [2,1,3] just random
    # random.permutation(5) --> [1,0,4,3,2]
    # In this case, we just create the random index
    # Then return pairs of (identity, positive example)
    # that each element in pairs in term of its index is randomly ordered.
    return np.array(listX1)[perm], np.array(listX2)[perm]

Xa_train, Xp_train = build_positive_pairs(range(0, num_classes))

# Gather the ids of all network traces that are used for training
# This just union of two sets set(A) | set(B)
all_traces_train_idx = list(set(Xa_train) | set(Xp_train))
print("X_train Anchor: ", Xa_train.shape)
print("X_train Positive: ", Xp_train.shape)

# Build a loss which doesn't take into account the y_true, as# Build
# we'll be passing only 0
def identity_loss(y_true, y_pred):
    return K.mean(y_pred - 0 * y_true)

# The real loss is here
def cosine_triplet_loss(X):
    _alpha = alpha_value
    positive_sim, negative_sim = X

    losses = K.maximum(0.0, negative_sim - positive_sim + _alpha)
    return K.mean(losses)

# ------------------- Hard Triplet Mining -----------
# Naive way to compute all similarities between all network traces.

def build_similarities(conv, all_imgs):
    embs = conv.predict(all_imgs)
    embs = embs / np.linalg.norm(embs, axis=-1, keepdims=True)
    all_sims = np.dot(embs, embs.T)
    return all_sims

def intersect(a, b):
    return list(set(a) & set(b))

def build_negatives(anc_idxs, pos_idxs, similarities, neg_imgs_idx, num_retries=50):
    # If no similarities were computed, return a random negative
    if similarities is None:
        return random.sample(neg_imgs_idx,len(anc_idxs))
    final_neg = []
    # for each positive pair
    for (anc_idx, pos_idx) in zip(anc_idxs, pos_idxs):
        anchor_class = id_to_classid[anc_idx]
        #positive similarity
        sim = similarities[anc_idx, pos_idx]
        # find all negatives which are semi(hard)
        possible_ids = np.where((similarities[anc_idx] + alpha_value) > sim)[0]
        possible_ids = intersect(neg_imgs_idx, possible_ids)
        appended = False
        for iteration in range(num_retries):
            if len(possible_ids) == 0:
                break
            idx_neg = random.choice(possible_ids)
            if id_to_classid[idx_neg] != anchor_class:
                final_neg.append(idx_neg)
                appended = True
                break
        if not appended:
            final_neg.append(random.choice(neg_imgs_idx))
    return final_neg


class SemiHardTripletGenerator():
    def __init__(self, Xa_train, Xp_train, batch_size, all_traces, neg_traces_idx, conv):
        self.batch_size = batch_size

        self.traces = all_traces
        self.Xa = Xa_train
        self.Xp = Xp_train
        self.cur_train_index = 0
        self.num_samples = Xa_train.shape[0]
        self.neg_traces_idx = neg_traces_idx
        self.all_anchors = list(set(Xa_train))
        self.mapping_pos = {v: k for k, v in enumerate(self.all_anchors)}
        self.mapping_neg = {k: v for k, v in enumerate(self.neg_traces_idx)}
        if conv:
            self.similarities = build_similarities(conv, self.traces)
        else:
            self.similarities = None

    def next_train(self):
        while 1:
            self.cur_train_index += self.batch_size
            if self.cur_train_index >= self.num_samples:
                self.cur_train_index = 0

            # fill one batch
            traces_a = self.Xa[self.cur_train_index:self.cur_train_index + self.batch_size]
            traces_p = self.Xp[self.cur_train_index:self.cur_train_index + self.batch_size]
            traces_n = build_negatives(traces_a, traces_p, self.similarities, self.neg_traces_idx)

            yield ([self.traces[traces_a],
                    self.traces[traces_p],
                    self.traces[traces_n]],
                   np.zeros(shape=(traces_a.shape[0]))
                   )

# Training the Triplet Model
shared_conv2 = DF(input_shape=(5000,1), emb_size=emb_size)

anchor = Input((5000, 1), name='anchor')
positive = Input((5000, 1), name='positive')
negative = Input((5000, 1), name='negative')

a = shared_conv2(anchor)
p = shared_conv2(positive)
n = shared_conv2(negative)

# The Dot layer in Keras now supports built-in Cosine similarity using the normalize = True parameter.
# From the Keras Docs:
# keras.layers.Dot(axes, normalize=True)
# normalize: Whether to L2-normalize samples along the dot product axis before taking the dot product.
#  If set to True, then the output of the dot product is the cosine proximity between the two samples.
pos_sim = Dot(axes=-1, normalize=True)([a,p])
neg_sim = Dot(axes=-1, normalize=True)([a,n])

# customized loss
loss = Lambda(cosine_triplet_loss,
              output_shape=(1,))(
             [pos_sim,neg_sim])

model_triplet = Model(
    inputs=[anchor, positive, negative],
    outputs=loss)
print(model_triplet.summary())

opt = optimizers.gradient_descent_v2.SGD(learning_rate=0.001, decay=1e-6, momentum=0.9, nesterov=True)

model_triplet.compile(loss=identity_loss, optimizer=opt)

batch_size = batch_size_value
# At first epoch we don't generate hard triplets
gen_hard = SemiHardTripletGenerator(Xa_train, Xp_train, batch_size, all_traces, all_traces_train_idx, None)
nb_epochs = number_epoch
csv_logger = CSVLogger('/root/Lab/TP_Lab/log/Training_Log_%s.csv'%description, append=True, separator=';')
for epoch in range(nb_epochs):
    print("built new hard generator for epoch "+str(epoch)) #update the fit_generator
    model_triplet.fit_generator(generator=gen_hard.next_train(),
                    steps_per_epoch=Xa_train.shape[0] // batch_size,
                    epochs=1, verbose=1, callbacks=[csv_logger])
    gen_hard = SemiHardTripletGenerator(Xa_train, Xp_train, batch_size, all_traces, all_traces_train_idx, shared_conv2)
    #For no semi-hard_triplet
    #gen_hard = HardTripletGenerator(Xa_train, Xp_train, batch_size, all_traces, all_traces_train_idx, None)
    shared_conv2.save('/root/Lab/TP_Lab/trained_model/%s.h5'%description)
shared_conv2.save('/root/Lab/TP_Lab/trained_model/%s.h5'%description)


/root/dataset/dataset/extracted_AWF775/
with parameters, Alpha: 0.1, Batch_size: 128, Embedded_size: 64, Epoch_num: 30
Triplet_Model
number of classes: 775
Load traces with  (19375, 5000, 1)
Total size allocated on RAM :  96.875 MB
X_train Anchor:  (232500,)
X_train Positive:  (232500,)


2023-01-22 18:26:00.861912: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1532] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 22310 MB memory:  -> device: 0, name: NVIDIA GeForce RTX 3090, pci bus id: 0000:9c:00.0, compute capability: 8.6


Model: "model_1"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 anchor (InputLayer)            [(None, 5000, 1)]    0           []                               
                                                                                                  
 positive (InputLayer)          [(None, 5000, 1)]    0           []                               
                                                                                                  
 negative (InputLayer)          [(None, 5000, 1)]    0           []                               
                                                                                                  
 model (Functional)             (None, 64)           1369344     ['anchor[0][0]',                 
                                                                  'positive[0][0]',         

/tmp/ipykernel_3137/3399004279.py:266: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  model_triplet.fit_generator(generator=gen_hard.next_train(),
2023-01-22 18:26:04.395513: I tensorflow/stream_executor/cuda/cuda_dnn.cc:384] Loaded cuDNN version 8101
2023-01-22 18:26:07.033416: I tensorflow/stream_executor/cuda/cuda_blas.cc:1786] TensorFloat-32 will be used for the matrix multiplication. This will only be logged once.


606/606 [==============================] - 2s 3ms/step
built new hard generator for epoch 1
606/606 [==============================] - 2s 3ms/step
built new hard generator for epoch 2
606/606 [==============================] - 2s 3ms/step
built new hard generator for epoch 3
606/606 [==============================] - 2s 3ms/step
built new hard generator for epoch 4
606/606 [==============================] - 1s 2ms/step
built new hard generator for epoch 5
606/606 [==============================] - 2s 3ms/step
built new hard generator for epoch 6
1679/1816 [==========================>...] - ETA: 17s - loss: 0.0500